# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 

from recfldtkn.ckpd_obs import Ckpd_ObservationS
from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_CGMGPT',
 'MODEL_TASK': 'Model_CGMGPT',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/GoogleDrive-junjie.luo.jhu@gmail.com/Shared '
                   'drives/CDHAI-WellDoc/2024-WellDocTest-SPACE/_WellDoc-AI-CGMGPT-WorkSpace'}


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools


####################
TriggerCaseMethod = 'CGM5MinEntry'
####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'CGM5Min',
 'case_id_columns': ['PID', 'ObsDT'],
 'special_columns': ['PID', 'DT_s'],
 'convert_TriggerEvent_to_Caseset': <function CGM5MinEntry.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [7]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['Ckpd_ObservationS'] = Ckpd_ObservationS
print(cohort_args)
print(TriggerCasePath)

df_case = pd.read_pickle(TriggerCasePath) # (1000)
df_case = df_case.sample(100, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 100
})


## [Step 2] RO: Record Observation & CasePhi


In [8]:
###########################
ROName_List = ['CGM5Min-Bf1M-N2Cin1']
name_CasePhi = 'AggMeanFeat'
###########################

In [9]:
from recfldtkn.obsname import convert_RONameList_to_COName

CaseObsName = convert_RONameList_to_COName(ROName_List, name_CasePhi)
print(CaseObsName)

ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat


In [10]:
# part of fetch_caseobs_Phi_tools
CO_Folder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CO_Folder): os.makedirs(CO_Folder)
print(CO_Folder)

../_Data/2-Data_CaseObs/ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [11]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1002840, 1002580, 1004670, 1003564, 1003011], 'ObsDT': [Timestamp('2021-12-07 13:50:00'), Timestamp('2021-11-10 12:15:00'), Timestamp('2021-11-13 06:40:00'), Timestamp('2021-09-15 13:35:00'), Timestamp('2021-10-17 17:15:00')]}


In [12]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [13]:
case_example = idx_to_examples[1]
case_example

{'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}

## [Step 2] get_caseobs_id

In [14]:
import inspect
#################################################
def get_CO_id(case_example, CaseObsName):
    PIDValue = case_example['PID']
    ObsDTValue = case_example['ObsDT'].isoformat()
    return f'{PIDValue}&{ObsDTValue}'

get_CO_id.fn_string = inspect.getsource(get_CO_id)
#################################################

In [15]:
caseobs_id = get_CO_id(case_example, CaseObsName)
caseobs_id # CO_id

'1002580&2021-11-10T12:15:00'



## [Step 3] RO List and CasePhi

In [16]:
name_CasePhi 

'AggMeanFeat'

In [17]:
## we need to prepare these five arguments.
# CaseTkn     # <------- we have this.
# cohort_args # <------- we have this.

# RecObs_Name 
# rec_args
# column_names # attribute name of R^recordname

In [18]:
######### RecObs_Name
print(ROName_List) # RO_List
# Let's focus on one RO
ROName = ROName_List[0]
print(ROName)

['CGM5Min-Bf1M-N2Cin1']
CGM5Min-Bf1M-N2Cin1


In [19]:
# RO = 'Rx-Bf2M-InsCate'
# RO = 'Rx-Bf2M'
# RO = 'P-Zip3DemoNume'

## [Step4] RO Information

In [20]:
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(ROName)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

CGM5Min Bf1M N2Cin1


In [21]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
[i for i in rec_args]

CGM5Min


['CohortInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'DTCols',
 'RecDT',
 'ParentRecName',
 'attr_cols',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'pypath',
 'recfldtkn_config_path',
 'yaml_file_path']

## [Step 5] Load ds_rec

In [22]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)
# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [23]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PatientID',
 'CGM5MinID',
 'DT_s',
 'BGValue',
 'CGM5Min-N2Cin1Tkn_wgt',
 'CGM5Min-N2Cin1Tkn_tknidx']

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [24]:
ROName

'CGM5Min-Bf1M-N2Cin1'

In [25]:
from recfldtkn.obsname import parse_RecObsName
# RecName = RecObs_Name.split('-')[0]
base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
base_columns

['PID', 'CGM5MinID', 'DT_s']

In [26]:

RecName = parse_RecObsName(ROName)['RecName']
FldName = parse_RecObsName(ROName)['FldName']
RecFldTknName = RecName + '-' + FldName + 'Tkn'
# rec_args['FldTknInfo'][RecFldTknName]
RecFldTknName

'CGM5Min-N2Cin1Tkn'

In [27]:
val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']

In [28]:
final_cols = base_columns + val_cols
final_cols

['PID', 'CGM5MinID', 'DT_s', 'BGValue']

In [29]:
###############################
def get_selected_columns(ROName, column_names, cohort_args, rec_args, CaseTkn):
    from recfldtkn.obsname import parse_RecObsName
    RecName = parse_RecObsName(ROName)['RecName']
    FldName = parse_RecObsName(ROName)['FldName']
    base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
    
    FldName_cols = [i for i in column_names if FldName in i]
    if len(FldName_cols) == 0: 
        parse_RecObsName(ROName)['FldName']
        RecFldTknName = RecName + '-' + FldName + 'Tkn'
        # rec_args['FldTknInfo'][RecFldTknName]
        
        val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']
        final_cols = base_columns + val_cols
    else:
        final_cols = base_columns + FldName_cols
    return final_cols

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [30]:
RecName

'CGM5Min'

In [31]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?
get_selected_columns(ROName, column_names, cohort_args, rec_args, name_CasePhi)

['PID',
 'CGM5MinID',
 'DT_s',
 'CGM5Min-N2Cin1Tkn_wgt',
 'CGM5Min-N2Cin1Tkn_tknidx']

In [32]:
print(ROName_List)
print(name_CasePhi)
print(get_selected_columns)

['CGM5Min-Bf1M-N2Cin1']
AggMeanFeat
<function get_selected_columns at 0x7f81082acca0>


In [33]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
ROName_to_ROInfo = get_RecObsName_to_RecObsInfo(ROName_List, 
                                                name_CasePhi, 
                                                get_selected_columns,
                                                cohort_args, 
                                                cohort_args['Ckpd_ObservationS'], 
                           
                                                record_to_ds_rec, 
                                                record_to_ds_rec_info)
    
ROName = ROName_List[0] # Rx-Bf2M  
ROInfo = ROName_to_ROInfo[ROName] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)

[INFO:2024-03-03 01:18:14,218:(configfn.py@110 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline/config_recfldtkn/Record/CGM5Min.yaml


In [34]:
[i for i in ROInfo]
# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'fldtkn_args',
 'fld_tokenizer_fn',
 'run_fldtkn_on_the_fly',
 'ds_rec',
 'df_rec_info']

In [35]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 21215912
})

## [Step 7] get_CO_vocab

In [36]:
def get_CO_vocab(RecObsName_to_RecObsInfo):
    # phi_list
    RecFldName_list = list(set([RecObsInfo['RecName'] + '-' +RecObsInfo['FldName'] 
                                for RecObsName, RecObsInfo in RecObsName_to_RecObsInfo.items()
                                if RecObsInfo['FldName'] is not None ]
                                ))
    
    # print('RecFldName_list----->', RecFldName_list)
    # for current version: for any Phi, we only process one or zero phi. 
    assert len(RecFldName_list) <= 1

    ############################ tkn 
    RecObsName = [i for i in RecObsName_to_RecObsInfo][0]
    RecObsInfo = RecObsName_to_RecObsInfo[RecObsName]
    idx2tkn = RecObsInfo['FldIdx2Tkn'] + ['NoObs']   
    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab = {}
    CaseTknVocab['tid'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    CaseTknVocab['wgt'] = {}
    ############################
    
    return CaseTknVocab

get_CO_vocab.fn_string = inspect.getsource(get_CO_vocab)

In [37]:
CO_vocab = get_CO_vocab(ROName_to_ROInfo)
pprint(CO_vocab, sort_dicts=False)

{'tid': {'tid2tkn': {0: 'unk',
                     1: 'BGValue:1~2',
                     2: 'BGValue:2~3',
                     3: 'BGValue:3~4',
                     4: 'BGValue:4~5',
                     5: 'BGValue:5~6',
                     6: 'BGValue:6~7',
                     7: 'BGValue:7~8',
                     8: 'BGValue:8~9',
                     9: 'BGValue:9~10',
                     10: 'BGValue:10~11',
                     11: 'BGValue:11~12',
                     12: 'BGValue:12~13',
                     13: 'BGValue:13~14',
                     14: 'BGValue:14~15',
                     15: 'BGValue:15~16',
                     16: 'BGValue:16~17',
                     17: 'BGValue:17~18',
                     18: 'BGValue:18~19',
                     19: 'BGValue:19~20',
                     20: 'BGValue:20~21',
                     21: 'BGValue:21~22',
                     22: 'BGValue:22~23',
                     23: 'BGValue:23~24',
                     24: 'BGV


## [Step 8] Fetch ds_rec

In [38]:
case_example = idx_to_examples[4]
case_example

{'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}

In [39]:
ROName

'CGM5Min-Bf1M-N2Cin1'

In [40]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')},
 1: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')},
 2: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')},
 3: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')},
 4: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17:15:00')}}


In [41]:
for ROName, ROInfo in ROName_to_ROInfo.items():
    print(ROName, ':', [i for i in ROInfo])

CGM5Min-Bf1M-N2Cin1 : ['rec_args', 'RecName', 'CkpdName', 'CkpdInfo', 'FldName', 'FldTknName', 'FldIdx2Tkn', 'fldtkn_args', 'fld_tokenizer_fn', 'run_fldtkn_on_the_fly', 'ds_rec', 'df_rec_info']


In [42]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import CaseObserverTransformer

get_Record_P = CaseObserverTransformer.get_Record_P
RecName_to_REC_P = get_Record_P(idx_to_examples, ROName_to_ROInfo)
for RecName, REC_P in RecName_to_REC_P.items():
    print(f'============ {RecName} ============')
    pprint(REC_P)

============ CGM5Min ============
{1002580: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 35212
}),
 1002840: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 47367
}),
 1003011: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 51200
}),
 1003564: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 39938
}),
 1004670: Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 20889
})}


## [Step 9] Fetch ROdf: $R_i^{recname}$

In [43]:
get_idx_to_RecObsName_to_RecObsDS = CaseObserverTransformer.get_idx_to_RecObsName_to_RecObsDS
idx_to_ROName_to_ROds = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, ROName_to_ROInfo, RecName_to_REC_P)

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    print('ROdf:', idx_to_ROName_to_ROds[idx]) # R_i^{RecNum, Ckpd}
    print('====================')


idx: 0
case: {'PID': 1002840, 'ObsDT': Timestamp('2021-12-07 13:50:00')}
ROdf: {'CGM5Min-Bf1M-N2Cin1': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8512
})}

idx: 1
case: {'PID': 1002580, 'ObsDT': Timestamp('2021-11-10 12:15:00')}
ROdf: {'CGM5Min-Bf1M-N2Cin1': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8257
})}

idx: 2
case: {'PID': 1004670, 'ObsDT': Timestamp('2021-11-13 06:40:00')}
ROdf: {'CGM5Min-Bf1M-N2Cin1': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8139
})}

idx: 3
case: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')}
ROdf: {'CGM5Min-Bf1M-N2Cin1': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8463
})}

idx: 4
case: {'PID': 1003011, 'ObsDT': Timestamp('2021-10-17 17

In [44]:
[i for i in ROName_to_ROInfo]

['CGM5Min-Bf1M-N2Cin1']

In [45]:
CO_vocab

{'tid': {'tid2tkn': {0: 'unk',
   1: 'BGValue:1~2',
   2: 'BGValue:2~3',
   3: 'BGValue:3~4',
   4: 'BGValue:4~5',
   5: 'BGValue:5~6',
   6: 'BGValue:6~7',
   7: 'BGValue:7~8',
   8: 'BGValue:8~9',
   9: 'BGValue:9~10',
   10: 'BGValue:10~11',
   11: 'BGValue:11~12',
   12: 'BGValue:12~13',
   13: 'BGValue:13~14',
   14: 'BGValue:14~15',
   15: 'BGValue:15~16',
   16: 'BGValue:16~17',
   17: 'BGValue:17~18',
   18: 'BGValue:18~19',
   19: 'BGValue:19~20',
   20: 'BGValue:20~21',
   21: 'BGValue:21~22',
   22: 'BGValue:22~23',
   23: 'BGValue:23~24',
   24: 'BGValue:24~25',
   25: 'BGValue:25~26',
   26: 'BGValue:26~27',
   27: 'BGValue:27~28',
   28: 'BGValue:28~29',
   29: 'BGValue:29~30',
   30: 'BGValue:30~31',
   31: 'BGValue:31~32',
   32: 'BGValue:32~33',
   33: 'BGValue:33~34',
   34: 'BGValue:34~35',
   35: 'BGValue:35~36',
   36: 'BGValue:36~37',
   37: 'BGValue:37~38',
   38: 'BGValue:38~39',
   39: 'BGValue:39~40',
   40: 'BGValue:40~41',
   41: 'BGValue:41~42',
   42: 'BGV

In [46]:
idx = 3
case_example = idx_to_examples[idx]
print('case_example:', case_example)
ROName_to_ROds = idx_to_ROName_to_ROds[idx] # RO_to_ROdf # RO_ij
print('RO_to_ROdf (RO_ij):', ROName_to_ROds)
print([i for i in ROName_to_ROds])

case_example: {'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')}
RO_to_ROdf (RO_ij): {'CGM5Min-Bf1M-N2Cin1': Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8463
})}
['CGM5Min-Bf1M-N2Cin1']


# [Step 10]: Develop $\Phi$. 

In [47]:
case_example

{'PID': 1003564, 'ObsDT': Timestamp('2021-09-15 13:35:00')}

In [48]:
ROName_to_ROds

{'CGM5Min-Bf1M-N2Cin1': Dataset({
     features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
     num_rows: 8463
 })}

In [49]:
ROName_to_ROInfo

{'CGM5Min-Bf1M-N2Cin1': {'rec_args': {'CohortInfo': {'RawData2022_CGM': {'TableFile': {'raw_data_path': '../_Data/0-Data_Raw/RawData2022_CGM/05_02_2022_ElogBGEntry.csv',
      'RawRootID': 'PatientID',
      'RecNumColumn': 'ElogBGEntry',
      'raw_columns': ['PatientID',
       'BGEntryID',
       'ObservationDateTime',
       'EntryCreatedDateTime',
       'ObservationEntryDateTime',
       'ModifiedDateTime',
       'UserObservationDateTime',
       'TimezoneOffset',
       'Timezone',
       'BGValue',
       'ActualBGValue',
       'ExternalSourceID',
       'MeterType']}},
    'RawData2023_CVSTDCAug': {'TableFile': {'raw_data_path': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/08_23_2023_ElogBGEntry.csv',
      'RawRootID': 'PatientID',
      'RecNumColumn': 'ElogBGEntry',
      'raw_columns': ['PatientID',
       'BGEntryID',
       'ObservationDateTime',
       'EntryCreatedDateTime',
       'ObservationEntryDateTime',
       'ModifiedDateTime',
       'UserObservationDateTime',

In [50]:
CO_vocab

{'tid': {'tid2tkn': {0: 'unk',
   1: 'BGValue:1~2',
   2: 'BGValue:2~3',
   3: 'BGValue:3~4',
   4: 'BGValue:4~5',
   5: 'BGValue:5~6',
   6: 'BGValue:6~7',
   7: 'BGValue:7~8',
   8: 'BGValue:8~9',
   9: 'BGValue:9~10',
   10: 'BGValue:10~11',
   11: 'BGValue:11~12',
   12: 'BGValue:12~13',
   13: 'BGValue:13~14',
   14: 'BGValue:14~15',
   15: 'BGValue:15~16',
   16: 'BGValue:16~17',
   17: 'BGValue:17~18',
   18: 'BGValue:18~19',
   19: 'BGValue:19~20',
   20: 'BGValue:20~21',
   21: 'BGValue:21~22',
   22: 'BGValue:22~23',
   23: 'BGValue:23~24',
   24: 'BGValue:24~25',
   25: 'BGValue:25~26',
   26: 'BGValue:26~27',
   27: 'BGValue:27~28',
   28: 'BGValue:28~29',
   29: 'BGValue:29~30',
   30: 'BGValue:30~31',
   31: 'BGValue:31~32',
   32: 'BGValue:32~33',
   33: 'BGValue:33~34',
   34: 'BGValue:34~35',
   35: 'BGValue:35~36',
   36: 'BGValue:36~37',
   37: 'BGValue:37~38',
   38: 'BGValue:38~39',
   39: 'BGValue:39~40',
   40: 'BGValue:40~41',
   41: 'BGValue:41~42',
   42: 'BGV

In [51]:
ROName = list(ROName_to_ROds.keys())[0]
ROName


'CGM5Min-Bf1M-N2Cin1'

In [52]:
ROds   = ROName_to_ROds[ROName]
ROds

Dataset({
    features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
    num_rows: 8463
})

In [53]:
ROdf = ROds.to_pandas()
ROdf

,PID,CGM5MinID,DT_s,CGM5Min-N2Cin1Tkn_wgt,CGM5Min-N2Cin1Tkn_tknidx
0,1003564,1003564-6379,2021-08-16 13:35:00,[1],[361]
1,1003564,1003564-6380,2021-08-16 13:40:00,[1],[360]
2,1003564,1003564-6381,2021-08-16 13:45:00,[1],[355]
3,1003564,1003564-6382,2021-08-16 13:50:00,[1],[354]
4,1003564,1003564-6383,2021-08-16 13:55:00,[1],[354]
...,...,...,...,...,...
8458,1003564,1003564-14837,2021-09-15 13:15:00,[1],[249]
8459,1003564,1003564-14838,2021-09-15 13:20:00,[1],[246]
8460,1003564,1003564-14839,2021-09-15 13:25:00,[1],[243]
8461,1003564,1003564-14840,2021-09-15 13:30:00,[1],[239]


In [54]:
ROdf = ROds.to_pandas()
        
FldName = parse_RecObsName(ROName)['FldName']
FldNameTkn = FldName + 'Tkn'

In [55]:
key_list = [i for i in ROdf.columns if FldNameTkn in i]
key_list

['CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx']

In [56]:
tknidx_col = [i for i in key_list if '_tknidx' in i][0]
tknidx_col

'CGM5Min-N2Cin1Tkn_tknidx'

In [57]:
wgt_col = [i for i in key_list if '_wgt' in i][0]
wgt_col

'CGM5Min-N2Cin1Tkn_wgt'

In [58]:
from recfldtkn.obsname import parse_RecObsName
import itertools
tknidx_seq = list(itertools.chain(*ROdf[tknidx_col].to_list()))
tknidx_seq

[361,
 360,
 355,
 354,
 354,
 357,
 358,
 353,
 345,
 337,
 331,
 327,
 319,
 317,
 314,
 314,
 309,
 298,
 278,
 258,
 244,
 243,
 248,
 256,
 264,
 273,
 284,
 298,
 310,
 318,
 321,
 324,
 330,
 336,
 341,
 349,
 355,
 357,
 350,
 343,
 336,
 327,
 313,
 291,
 265,
 243,
 231,
 230,
 232,
 233,
 229,
 223,
 216,
 213,
 213,
 214,
 214,
 216,
 219,
 221,
 223,
 223,
 224,
 225,
 228,
 231,
 236,
 239,
 238,
 234,
 233,
 237,
 247,
 259,
 272,
 282,
 290,
 300,
 311,
 322,
 331,
 339,
 348,
 356,
 362,
 363,
 353,
 339,
 319,
 301,
 280,
 267,
 261,
 260,
 260,
 260,
 262,
 266,
 271,
 277,
 283,
 292,
 298,
 303,
 305,
 309,
 317,
 328,
 341,
 351,
 357,
 359,
 359,
 360,
 361,
 365,
 374,
 387,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 401,
 399,
 390,
 386,
 388,
 390,
 390,
 390,
 389,
 390,
 390,
 390,
 390,
 390,
 391,
 392

In [59]:
wgt_seq = list(itertools.chain(*ROdf[wgt_col].to_list()))
wgt_seq

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [60]:
df_feat = pd.DataFrame({'tknidx': tknidx_seq, 'wgt': wgt_seq})
df_feat

,tknidx,wgt
0,361,1
1,360,1
2,355,1
3,354,1
4,354,1
...,...,...
8458,249,1
8459,246,1
8460,243,1
8461,239,1


In [61]:
df_feat['tknidx'].value_counts()

tknidx
401    277
210     52
225     48
267     47
193     47
      ... 
55       1
60       1
63       1
66       1
52       1
Name: count, Length: 347, dtype: int64

In [62]:
df_feat = df_feat.groupby('tknidx').sum().reset_index()
df_feat

,tknidx,wgt
0,39,3
1,52,1
2,53,1
3,54,2
4,55,1
...,...,...
342,397,6
343,398,4
344,399,6
345,400,4


In [63]:
tid = df_feat['tknidx'].to_list()
tid

[39,
 52,
 53,
 54,
 55,
 56,
 57,
 59,
 60,
 62,
 63,
 64,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 

In [64]:
wgt = df_feat['wgt'].to_list()
wgt

[3,
 1,
 1,
 2,
 1,
 1,
 3,
 2,
 1,
 3,
 1,
 1,
 1,
 3,
 1,
 3,
 4,
 7,
 6,
 14,
 9,
 6,
 4,
 7,
 6,
 11,
 5,
 4,
 9,
 6,
 4,
 8,
 7,
 8,
 8,
 13,
 11,
 10,
 12,
 12,
 18,
 19,
 13,
 14,
 17,
 4,
 19,
 22,
 18,
 21,
 21,
 18,
 20,
 21,
 20,
 27,
 17,
 14,
 24,
 15,
 23,
 30,
 28,
 23,
 25,
 22,
 23,
 23,
 20,
 20,
 19,
 19,
 26,
 16,
 25,
 26,
 22,
 26,
 21,
 30,
 23,
 27,
 26,
 25,
 25,
 24,
 25,
 35,
 32,
 30,
 30,
 25,
 31,
 22,
 26,
 17,
 31,
 25,
 28,
 36,
 20,
 32,
 35,
 36,
 31,
 29,
 42,
 29,
 33,
 42,
 27,
 39,
 26,
 25,
 33,
 30,
 22,
 33,
 21,
 25,
 26,
 21,
 36,
 32,
 32,
 34,
 33,
 35,
 39,
 31,
 29,
 33,
 41,
 30,
 31,
 33,
 31,
 34,
 47,
 39,
 34,
 41,
 36,
 28,
 30,
 34,
 25,
 34,
 25,
 39,
 28,
 39,
 32,
 36,
 41,
 52,
 37,
 42,
 32,
 41,
 30,
 43,
 36,
 42,
 41,
 44,
 43,
 42,
 40,
 36,
 48,
 44,
 32,
 38,
 44,
 40,
 39,
 27,
 33,
 36,
 28,
 35,
 41,
 38,
 25,
 25,
 39,
 30,
 43,
 37,
 24,
 37,
 29,
 33,
 33,
 27,
 36,
 34,
 26,
 29,
 35,
 37,
 28,
 37,
 27,
 34,
 30,

In [65]:
CO = {'tid': tid, 'wgt': wgt}
CO


{'tid': [39,
  52,
  53,
  54,
  55,
  56,
  57,
  59,
  60,
  62,
  63,
  64,
  66,
  67,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  

In [66]:
def fn_CasePhi(case_example, 
               ROName_to_ROds, 
               ROName_to_ROInfo, 
               CO_vocab):
    
    from recfldtkn.obsname import parse_RecObsName
    import itertools
    
    # input: RecObsName_to_RecObsDS, RecObsName_to_RecObsInfo
    # output: CaseObservation
    assert len(ROName_to_ROds) == 1
    
    # RO
    ROName = list(ROName_to_ROds.keys())[0]
    # for RecObsName in RecObsName_to_RecObsDS:
    
    #############################################
    ROds   = ROName_to_ROds[ROName] # dataframe: RecObsName: Rx-bf24.. RecObsDS: the df: record collection
    ROInfo = ROName_to_ROds[ROName]
    
    d = {}
    if ROds is not None:
        ROdf = ROds.to_pandas()
        
        FldName = parse_RecObsName(ROName)['FldName']
        FldNameTkn = FldName + 'Tkn'
        
        key_list = [i for i in ROdf.columns if FldNameTkn in i]
        tknidx_col = [i for i in key_list if '_tknidx' in i][0]
        wgt_col = [i for i in key_list if '_wgt' in i][0]
        #############################################
        tknidx_seq = list(itertools.chain(*ROdf[tknidx_col].to_list()))
        wgt_seq = list(itertools.chain(*ROdf[wgt_col].to_list()))
        df_feat = pd.DataFrame({'tknidx': tknidx_seq, 'wgt': wgt_seq})
        df_feat = df_feat.groupby('tknidx').mean().reset_index()
        tid = df_feat['tknidx'].to_list()
        wgt = df_feat['wgt'].to_list()
        #############################################
        CO = {'tid': tid, 'wgt': wgt}

    else:
        tkn2tid = CO_vocab['tkn']['tkn2tid']
        # d['NoObs'] = 1
        tkn = ['NoObs']
        tid = [tkn2tid[i] for i in tkn]
        wgt = [1]
        CO = {'tid': tid, 'wgt': wgt}
    #############################################
    return CO

fn_CasePhi.fn_string = inspect.getsource(fn_CasePhi)

In [67]:
ROName_to_ROds

{'CGM5Min-Bf1M-N2Cin1': Dataset({
     features: ['PID', 'CGM5MinID', 'DT_s', 'CGM5Min-N2Cin1Tkn_wgt', 'CGM5Min-N2Cin1Tkn_tknidx'],
     num_rows: 8463
 })}

In [68]:
fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab)

{'tid': [39,
  52,
  53,
  54,
  55,
  56,
  57,
  59,
  60,
  62,
  63,
  64,
  66,
  67,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  

In [69]:
CO

{'tid': [39,
  52,
  53,
  54,
  55,
  56,
  57,
  59,
  60,
  62,
  63,
  64,
  66,
  67,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  

In [70]:
CO_vocab

{'tid': {'tid2tkn': {0: 'unk',
   1: 'BGValue:1~2',
   2: 'BGValue:2~3',
   3: 'BGValue:3~4',
   4: 'BGValue:4~5',
   5: 'BGValue:5~6',
   6: 'BGValue:6~7',
   7: 'BGValue:7~8',
   8: 'BGValue:8~9',
   9: 'BGValue:9~10',
   10: 'BGValue:10~11',
   11: 'BGValue:11~12',
   12: 'BGValue:12~13',
   13: 'BGValue:13~14',
   14: 'BGValue:14~15',
   15: 'BGValue:15~16',
   16: 'BGValue:16~17',
   17: 'BGValue:17~18',
   18: 'BGValue:18~19',
   19: 'BGValue:19~20',
   20: 'BGValue:20~21',
   21: 'BGValue:21~22',
   22: 'BGValue:22~23',
   23: 'BGValue:23~24',
   24: 'BGValue:24~25',
   25: 'BGValue:25~26',
   26: 'BGValue:26~27',
   27: 'BGValue:27~28',
   28: 'BGValue:28~29',
   29: 'BGValue:29~30',
   30: 'BGValue:30~31',
   31: 'BGValue:31~32',
   32: 'BGValue:32~33',
   33: 'BGValue:33~34',
   34: 'BGValue:34~35',
   35: 'BGValue:35~36',
   36: 'BGValue:36~37',
   37: 'BGValue:37~38',
   38: 'BGValue:38~39',
   39: 'BGValue:39~40',
   40: 'BGValue:40~41',
   41: 'BGValue:41~42',
   42: 'BGV

## Save Files

In [71]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CO_id, get_selected_columns, get_CO_vocab, fn_CasePhi]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casephi', f'phi_{name_CasePhi}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [72]:
CaseObsName

'ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat'

In [73]:
from recfldtkn.observer import get_CaseObsInfo_for_a_CaseObsName

CaseObsInfo = get_CaseObsInfo_for_a_CaseObsName(CaseObsName,
                                                SPACE, 
                                                cohort_args, 
                                                record_to_ds_rec = {}, 
                                                record_to_ds_rec_info = {})
[i for i in CaseObsInfo]

[INFO:2024-03-03 01:19:19,537:(configfn.py@110 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline/config_recfldtkn/Record/CGM5Min.yaml


['RecObsName_List',
 'name_CasePhi',
 'get_selected_columns',
 'CaseObsName',
 'ROName_to_ROInfo',
 'fn_CasePhi',
 'get_CO_id',
 'CO_Folder',
 'CO_vocab']

In [75]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


ro_to_ROName = {} # leave it empty for now 
ROName_to_ROInfo = CaseObsInfo['ROName_to_ROInfo']
name_CasePhi = CaseObsInfo['name_CasePhi']
fn_CasePhi = CaseObsInfo['fn_CasePhi']
CO_vocab = CaseObsInfo['CO_vocab']
get_CO_id = CaseObsInfo['get_CO_id']
CO_Folder = CaseObsInfo['CO_Folder']
df_case = None 
use_CO_from_disk = False
batch_size = CaseObsInfo.get('batch_size', 1000)

fn_caseobs_Phi = CaseObserverTransformer(ro_to_ROName,
                                         ROName_to_ROInfo, 
                                         name_CasePhi, 
                                         fn_CasePhi, 
                                         CO_vocab, 
                                         get_CO_id,
                                         CO_Folder, 
                                         df_case,
                                         use_CO_from_disk)

# Check Cache Functions

In [76]:
fn_caseobs_Phi.new_COs

{}

In [77]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:04.399077
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt'],
    num_rows: 100
})


In [78]:
print(len(fn_caseobs_Phi.new_COs))

100


In [79]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Elipse Time:  0:00:00.013651
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt'],
    num_rows: 100
})


In [80]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int])
print(CO_vocab)

ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat
62
{'ObsDT': Timestamp('2021-12-07 20:05:00'),
 'PID': 1004917,
 'tid': [66,
         67,
         68,
         69,
         70,
         71,
         72,
         75,
         76,
         77,
         78,
         79,
         80,
         81,
         82,
         83,
         84,
         85,
         86,
         87,
         88,
         89,
         90,
         91,
         92,
         93,
         94,
         95,
         96,
         97,
         98,
         99,
         100,
         101,
         102,
         103,
         104,
         105,
         106,
         107,
         108,
         109,
         110,
         111,
         112,
         113,
         114,
         115,
         116,
         117,
         118,
         119,
         120,
         121,
         122,
         123,
         124,
         125,
         126,
         127,
         128,
         129,
         130,
         131,
         132,
         133,
      

In [81]:
ds_casetkn[1]

{'PID': 1002580,
 'ObsDT': Timestamp('2021-11-10 12:15:00'),
 'tid': [44,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  18

## Save

In [82]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [83]:
CO_Folder_data = fn_caseobs_Phi.CO_Folder_data 
CO_Folder_data

'../_Data/2-Data_CaseObs/ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat/data'

In [84]:
fn_caseobs_Phi.save_new_COs_to_disk(CO_Folder_data)

[INFO:2024-03-03 01:20:29,183:(observer.py@630 recfldtkn.observer)]: use_CO_from_disk is false and empty new_COs


In [85]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [86]:
fn_caseobs_Phi.ds_CO_data

# Save Vocab

In [90]:
CO_vocab = fn_caseobs_Phi.CO_vocab
CO_vocab

{'tid': {'tid2tkn': {0: 'unk',
   1: 'BGValue:1~2',
   2: 'BGValue:2~3',
   3: 'BGValue:3~4',
   4: 'BGValue:4~5',
   5: 'BGValue:5~6',
   6: 'BGValue:6~7',
   7: 'BGValue:7~8',
   8: 'BGValue:8~9',
   9: 'BGValue:9~10',
   10: 'BGValue:10~11',
   11: 'BGValue:11~12',
   12: 'BGValue:12~13',
   13: 'BGValue:13~14',
   14: 'BGValue:14~15',
   15: 'BGValue:15~16',
   16: 'BGValue:16~17',
   17: 'BGValue:17~18',
   18: 'BGValue:18~19',
   19: 'BGValue:19~20',
   20: 'BGValue:20~21',
   21: 'BGValue:21~22',
   22: 'BGValue:22~23',
   23: 'BGValue:23~24',
   24: 'BGValue:24~25',
   25: 'BGValue:25~26',
   26: 'BGValue:26~27',
   27: 'BGValue:27~28',
   28: 'BGValue:28~29',
   29: 'BGValue:29~30',
   30: 'BGValue:30~31',
   31: 'BGValue:31~32',
   32: 'BGValue:32~33',
   33: 'BGValue:33~34',
   34: 'BGValue:34~35',
   35: 'BGValue:35~36',
   36: 'BGValue:36~37',
   37: 'BGValue:37~38',
   38: 'BGValue:38~39',
   39: 'BGValue:39~40',
   40: 'BGValue:40~41',
   41: 'BGValue:41~42',
   42: 'BGV

In [91]:
CO_Folder_vocab = fn_caseobs_Phi.CO_Folder_vocab
CO_Folder_vocab

'../_Data/2-Data_CaseObs/ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat/vocab.p'

In [92]:
df_Vocab = pd.DataFrame({CaseObsName: CO_vocab})
df_Vocab.to_pickle(CO_Folder_vocab)
df_Vocab

,ro.CGM5Min-Bf1M-N2Cin1_ct.AggMeanFeat
tid,"{'tid2tkn': {0: 'unk', 1: 'BGValue:1~2', 2: 'B..."
wgt,{}
